In [1]:
import pandas as pd
from src.train import train_loop, test_loop
from src.model import CNNLSTMModel
from src.dataset import InverterTimeSeriesDataset
from src.preprocess import *
from src.visualize import *
import torch

inverter_data = load_parquet_data('data/inverter_data')
failure_sessions = load_failure_sessions('data/failure_sessions.csv', min_days=3)

Loaded 15 parquet files → 6126272 rows
Kept 61 sessions longer than 3 days


In [18]:
import pandas as pd
import numpy as np

def find_time_gaps(df: pd.DataFrame,
                   time_col: str = "event_local_time",
                   device_col: str | None = "device_name",
                   freq: str = "5min") -> pd.DataFrame:
    df = df.copy()
    df[time_col] = pd.to_datetime(df[time_col], errors="coerce")

    expected = pd.to_timedelta(freq)
    expected_min = expected / pd.Timedelta(minutes=1)

    def _calc(g: pd.DataFrame) -> pd.DataFrame:
        g = g.sort_values(time_col)
        if len(g) < 2:
            return pd.DataFrame(columns=["prev_time","curr_time","gap_minutes","gap_type",
                                         "expected_minutes","missing_points_estimate"])
        diff = g[time_col].diff()

        out = pd.DataFrame({
            "prev_time": g[time_col].shift(1),
            "curr_time": g[time_col],
            "gap_minutes": diff / pd.Timedelta(minutes=1)
        }).iloc[1:]  # 第一列無前一筆

        gm = out["gap_minutes"]

        cond_missing  = gm >  expected_min
        cond_duplicate = gm == 0
        cond_overlap   = (gm < expected_min) & (gm > 0)  # 比預期更密
        cond_outorder  = gm < 0                          # 倒退（理應不會出現在已排序，但保險）

        # 依優先序標記（缺口 > 倒退/過密 > 重複）
        gap_type = np.where(cond_missing, "missing",
                    np.where(cond_outorder | cond_overlap, "overlap_or_out_of_order",
                    np.where(cond_duplicate, "duplicate", "ok")))

        out["gap_type"] = gap_type
        out = out[out["gap_type"] != "ok"]

        out["expected_minutes"] = float(expected_min)
        out["missing_points_estimate"] = np.where(
            out["gap_type"] == "missing",
            (out["gap_minutes"] / expected_min - 1).round().astype(int),
            0
        )
        return out

    if device_col and device_col in df.columns:
        parts = []
        for dev, g in df.groupby(device_col, sort=False):
            tmp = _calc(g)
            if not tmp.empty:
                tmp.insert(0, device_col, dev)
            parts.append(tmp)
        gaps = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame()
    else:
        gaps = _calc(df)

    return gaps


def is_continuous(df: pd.DataFrame,
                  time_col: str = "event_local_time",
                  device_col: str | None = "device_name",
                  freq: str = "5min") -> bool:
    return find_time_gaps(df, time_col=time_col, device_col=device_col, freq=freq).empty


In [20]:
time_gap = find_time_gaps(inverter_data, time_col='event_local_time', device_col='device_name', freq='5min')
time_gap.to_csv('data/time_gap.csv', index=False)

In [2]:
# some important parameters
pre_day = 30
window_size = 12*24 # 5 minutes * 12 * 24 = 1 day
stride = 12 # 1 hour stride

feature_cols = [
    #"metric.STATUS_AC_MOD_ADMISSION_TEMP.MEASURED",  # ambient temperature
    #"metric.STATUS_INTERNAL_TEMP.MEASURED",          # internal temperature
    "metric.AC_VOLTAGE_AB.MEASURED",                 # AC voltage
    "metric.AC_VOLTAGE_BC.MEASURED",                 # AC voltage
    "metric.AC_VOLTAGE_CA.MEASURED",                 # AC voltage
    "metric.DC_VOLTAGE.MEASURED"                     # DC voltage
]

exclude_periods = [
    [pd.Timestamp('2021-01-01'), pd.Timestamp('2021-12-23')], # data collection issue
    [pd.Timestamp('2023-02-23'), pd.Timestamp('2023-08-26')] # anomalies in the data
]

In [ ]:
#visualize_hourly_mean_values(inverter_data, failure_sessions, feature_cols, 'visualization/raw_data/')

In [3]:
print("inverter_data shape:", inverter_data.shape)
excluded_data = exclude_periods_from_data(inverter_data, exclude_periods)
print("Excluded data shape:", excluded_data.shape)

inverter_data shape: (6126272, 59)
Excluded data shape: (5172608, 59)


In [24]:
selected_data = excluded_data[['event_local_time', 'device_name'] + feature_cols]

In [25]:
visualize_hourly_mean_values(selected_data, failure_sessions, feature_cols, 'visualization/filtered_data/')

Visualization saved for device: INV 51 at visualization/filtered_data//INV 51.html
Visualization saved for device: INV 52 at visualization/filtered_data//INV 52.html
Visualization saved for device: INV 53 at visualization/filtered_data//INV 53.html
Visualization saved for device: INV 54 at visualization/filtered_data//INV 54.html
Visualization saved for device: INV 55 at visualization/filtered_data//INV 55.html
Visualization saved for device: INV 56 at visualization/filtered_data//INV 56.html
Visualization saved for device: INV 57 at visualization/filtered_data//INV 57.html
Visualization saved for device: INV 58 at visualization/filtered_data//INV 58.html
Visualization saved for device: INV 59 at visualization/filtered_data//INV 59.html
Visualization saved for device: INV 60 at visualization/filtered_data//INV 60.html
Visualization saved for device: INV 61 at visualization/filtered_data//INV 61.html
Visualization saved for device: INV 62 at visualization/filtered_data//INV 62.html
Visu

In [ ]:
labeled_df = prepare_dataset(selected_data, failure_sessions, pre_days=pre_day)

Total pre-failure rows: 365836
Total rows: 5026798


In [56]:
def generate_missing_value_mask(df: pd.DataFrame, features_cols: list[str]) -> pd.DataFrame:
    # Step 1: 缺失 mask
    for col in features_cols:
        df[f"{col}_missing"] = df[col].isna().astype(int)
        
generate_missing_value_mask(labeled_df, feature_cols)

In [61]:
from typing import List

def missing_value_imputation(
    df: pd.DataFrame,
    feature_cols: List[str],
    time_col: str = "event_local_time",
    device_col: str = "device_name",
    short_gap_limit: int = 6,   # 5 分鐘資料 -> 6 筆 ≈ 30 分鐘內用插值
    long_fill_value: float = 0.0,
    add_missing_mask: bool = True,
) -> pd.DataFrame:
    """
    針對多裝置時間序列做缺失補值：
      1) 先產生 per-step 缺失 mask（可選）
      2) 每個裝置內，以時間排序後對 feature 做「時間型插值」(limit=short_gap_limit)
      3) 尚未補到的長缺失以指定值（預設 0）補齊

    參數：
      - df: 原始 DataFrame，需包含 time_col 與 device_col
      - feature_cols: 要補值的數值欄位
      - time_col: 時間欄位名稱（需可轉為 datetime）
      - device_col: 裝置欄位名稱
      - short_gap_limit: 連續缺失筆數在此上限以內使用插值
      - long_fill_value: 插值後仍為 NaN 的長缺失以此值補
      - add_missing_mask: 是否為每個 feature 產生 *_missing 的 0/1 mask 欄位

    回傳：
      - 完成補值與（可選）新增 mask 的 DataFrame
    """
    imputed_df = df.copy()

    # 確保時間欄為 datetime
    imputed_df[time_col] = pd.to_datetime(imputed_df[time_col], errors="coerce")

    # 需要的欄位存在性檢查
    missing_cols = [c for c in [time_col, device_col] + feature_cols if c not in imputed_df.columns]
    if missing_cols:
        raise KeyError(f"Columns not found in df: {missing_cols}")

    for device, device_data in imputed_df.groupby(device_col, sort=False):
        # 複製避免 SettingWithCopy
        block = device_data.loc[:, [time_col, device_col] + feature_cols].copy()
        # 記住原始索引以便放回
        block["_orig_idx"] = block.index

        # 產生 per-step 缺失 mask（基於原始缺失）
        if add_missing_mask:
            for col in feature_cols:
                imputed_df.loc[block["_orig_idx"], f"{col}_missing"] = block[col].isna().astype(int)

        # 依時間排序並以時間為索引做 time-based interpolate
        block = block.sort_values(time_col)
        block = block.set_index(time_col)

        # 僅對目標特徵做處理
        # 短缺失：時間插值（雙向皆可，避免前段或尾段全 NaN 無法補）
        block[feature_cols] = block[feature_cols].interpolate(
            method="time", limit=short_gap_limit
        ).interpolate(method="time", limit_direction="both")

        # 長缺失：仍為 NaN 的以指定值補齊
        block[feature_cols] = block[feature_cols].fillna(long_fill_value)

        # 還原索引與順序
        block = block.reset_index()
        block = block.set_index("_orig_idx").sort_index()

        # 寫回 imputed_df（僅覆蓋目標特徵欄）
        imputed_df.loc[block.index, feature_cols] = block[feature_cols].values

    return imputed_df


In [62]:
imputed_df = missing_value_imputation(labeled_df, feature_cols, time_col='event_local_time', device_col='device_name', short_gap_limit=6, long_fill_value=0.0, add_missing_mask=True)

In [63]:
train_df, test_df = train_test_split_on_time(imputed_df, 0.2)
val_df, test_df = train_test_split_on_time(test_df, 0.5)

Train set size: 4021439 Train set time range: 2021-12-24 00:00:00 to 2024-12-04 04:10:00
Test set size: 1005359 Test set time range: 2024-12-04 04:10:00 to 2025-07-23 23:35:00
Train set size: 502680 Train set time range: 2024-12-04 04:10:00 to 2025-03-23 07:15:00
Test set size: 502679 Test set time range: 2025-03-23 07:15:00 to 2025-07-23 23:35:00


In [64]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_df[feature_cols] = scaler.fit_transform(train_df[feature_cols])
val_df[feature_cols] = scaler.transform(val_df[feature_cols])
test_df[feature_cols] = scaler.transform(test_df[feature_cols])

In [66]:
train_df.columns

Index(['event_local_time', 'device_name', 'metric.AC_VOLTAGE_AB.MEASURED',
       'metric.AC_VOLTAGE_BC.MEASURED', 'metric.AC_VOLTAGE_CA.MEASURED',
       'metric.DC_VOLTAGE.MEASURED', 'label',
       'metric.AC_VOLTAGE_AB.MEASURED_missing',
       'metric.AC_VOLTAGE_BC.MEASURED_missing',
       'metric.AC_VOLTAGE_CA.MEASURED_missing',
       'metric.DC_VOLTAGE.MEASURED_missing'],
      dtype='object')

In [73]:
feature_cols = feature_cols+[col+'_missing' for col in feature_cols]

In [74]:
feature_cols

['metric.AC_VOLTAGE_AB.MEASURED',
 'metric.AC_VOLTAGE_BC.MEASURED',
 'metric.AC_VOLTAGE_CA.MEASURED',
 'metric.DC_VOLTAGE.MEASURED',
 'metric.AC_VOLTAGE_AB.MEASURED_missing',
 'metric.AC_VOLTAGE_BC.MEASURED_missing',
 'metric.AC_VOLTAGE_CA.MEASURED_missing',
 'metric.DC_VOLTAGE.MEASURED_missing']

In [ ]:
visualize_hourly_mean_values(train_df, failure_sessions, feature_cols, 'visualization/train_data/')

Visualization saved for device: INV 51 at visualization/train_data//INV 51.html
Visualization saved for device: INV 59 at visualization/train_data//INV 59.html
Visualization saved for device: INV 64 at visualization/train_data//INV 64.html
Visualization saved for device: INV 62 at visualization/train_data//INV 62.html
Visualization saved for device: INV 52 at visualization/train_data//INV 52.html
Visualization saved for device: INV 53 at visualization/train_data//INV 53.html
Visualization saved for device: INV 60 at visualization/train_data//INV 60.html
Visualization saved for device: INV 66 at visualization/train_data//INV 66.html
Visualization saved for device: INV 58 at visualization/train_data//INV 58.html
Visualization saved for device: INV 57 at visualization/train_data//INV 57.html
Visualization saved for device: INV 55 at visualization/train_data//INV 55.html
Visualization saved for device: INV 65 at visualization/train_data//INV 65.html
Visualization saved for device: INV 56 a

In [75]:
from torch.utils.data import DataLoader

train_ds = InverterTimeSeriesDataset(train_df, feature_cols, under_sample=True, window_size=window_size, stride=stride)
val_ds   = InverterTimeSeriesDataset(val_df,   feature_cols, window_size=window_size, stride=stride)
test_ds  = InverterTimeSeriesDataset(test_df,  feature_cols, window_size=window_size, stride=stride)

Processing devices: 100%|██████████| 16/16 [00:00<00:00, 47.35it/s]


In [79]:
train_ds.X.shape

torch.Size([44806, 288, 8])

In [69]:
pd.value_counts(train_ds.y.numpy()), pd.value_counts(val_ds.y.numpy()), pd.value_counts(test_ds.y.numpy())

(0.0    22403
 1.0    22403
 dtype: int64,
 0.0    40208
 1.0      928
 dtype: int64,
 0.0    34432
 1.0     6780
 dtype: int64)

In [80]:
batch_size = 2**10
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=6, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=6, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=6, pin_memory=True)

In [81]:
model = CNNLSTMModel(num_features=len(feature_cols), cnn_out_channels=64, lstm_hidden_size=128)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

In [82]:
train_loop(model, train_loader, val_loader, log_interval=10, num_epochs=10, optimizer=optimizer, criterion=criterion)

Model moved to cuda
[Epoch 1/10] Step 0/44 - Loss: 0.2505
[Epoch 1/10] Step 10/44 - Loss: 0.2401
[Epoch 1/10] Step 20/44 - Loss: 0.2394
[Epoch 1/10] Step 30/44 - Loss: 0.2402
[Epoch 1/10] Step 40/44 - Loss: 0.2360
🔁 Epoch 1 finished. Avg Train Loss: 0.2408
✅ Validation Loss: 0.1792 | Accuracy: 65.78%
[Epoch 2/10] Step 0/44 - Loss: 0.2401
[Epoch 2/10] Step 10/44 - Loss: 0.2313
[Epoch 2/10] Step 20/44 - Loss: 0.2364
[Epoch 2/10] Step 30/44 - Loss: 0.2370
[Epoch 2/10] Step 40/44 - Loss: 0.2634
🔁 Epoch 2 finished. Avg Train Loss: 0.2383
✅ Validation Loss: 0.1132 | Accuracy: 90.33%
[Epoch 3/10] Step 0/44 - Loss: 0.2444
[Epoch 3/10] Step 10/44 - Loss: 0.2349
[Epoch 3/10] Step 20/44 - Loss: 0.2359
[Epoch 3/10] Step 30/44 - Loss: 0.2443
[Epoch 3/10] Step 40/44 - Loss: 0.2317
🔁 Epoch 3 finished. Avg Train Loss: 0.2360
✅ Validation Loss: 0.1835 | Accuracy: 72.35%
[Epoch 4/10] Step 0/44 - Loss: 0.2341
[Epoch 4/10] Step 10/44 - Loss: 0.2313
[Epoch 4/10] Step 20/44 - Loss: 0.2261
[Epoch 4/10] Step 

In [85]:
train_loop(model, train_loader, val_loader, log_interval=10, num_epochs=10, optimizer=optimizer, criterion=criterion)

Model moved to cuda
[Epoch 1/10] Step 0/44 - Loss: 0.2199
[Epoch 1/10] Step 10/44 - Loss: 0.2196
[Epoch 1/10] Step 20/44 - Loss: 0.2184
[Epoch 1/10] Step 30/44 - Loss: 0.2258
[Epoch 1/10] Step 40/44 - Loss: 0.2263
🔁 Epoch 1 finished. Avg Train Loss: 0.2218
✅ Validation Loss: 0.1751 | Accuracy: 72.38%
[Epoch 2/10] Step 0/44 - Loss: 0.2257
[Epoch 2/10] Step 10/44 - Loss: 0.2180
[Epoch 2/10] Step 20/44 - Loss: 0.2349
[Epoch 2/10] Step 30/44 - Loss: 0.2206
[Epoch 2/10] Step 40/44 - Loss: 0.2181
🔁 Epoch 2 finished. Avg Train Loss: 0.2220
✅ Validation Loss: 0.1617 | Accuracy: 76.04%
[Epoch 3/10] Step 0/44 - Loss: 0.2303
[Epoch 3/10] Step 10/44 - Loss: 0.2072
[Epoch 3/10] Step 20/44 - Loss: 0.2188
[Epoch 3/10] Step 30/44 - Loss: 0.2208
[Epoch 3/10] Step 40/44 - Loss: 0.2165
🔁 Epoch 3 finished. Avg Train Loss: 0.2176
✅ Validation Loss: 0.1947 | Accuracy: 62.65%
[Epoch 4/10] Step 0/44 - Loss: 0.2300
[Epoch 4/10] Step 10/44 - Loss: 0.2187
[Epoch 4/10] Step 20/44 - Loss: 0.2287
[Epoch 4/10] Step 

In [90]:
torch.save(model.state_dict(), 'model/20_epochs.pth')

In [91]:
loaded_model = CNNLSTMModel(num_features=len(feature_cols), cnn_out_channels=64, lstm_hidden_size=128)
loaded_model.load_state_dict(torch.load('model/20_epochs.pth'))
loaded_model.eval()

CNNLSTMModel(
  (cnn): Sequential(
    (0): Conv1d(8, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3, inplace=False)
  )
  (lstm): LSTM(64, 128, batch_first=True)
  (classifier): Sequential(
    (0): Linear(in_features=128, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=32, out_features=1, bias=True)
    (4): Sigmoid()
  )
)

In [86]:
trues, predictions = test_loop(model, test_loader, device='cuda', criterion=criterion)

🔍 Test Loss: 0.2453 | Accuracy: 48.57%


In [87]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(trues, predictions , target_names=['Normal', 'Failure']))
print(confusion_matrix(trues, predictions ))

              precision    recall  f1-score   support

      Normal       0.85      0.47      0.60     34432
     Failure       0.17      0.57      0.27      6780

    accuracy                           0.49     41212
   macro avg       0.51      0.52      0.44     41212
weighted avg       0.74      0.49      0.55     41212

[[16143 18289]
 [ 2908  3872]]
